In [1]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [202]:
import pandas as pd
import numpy as np
import json
import os
from string import digits

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

from molmass import Formula

from main_functions import *

# Molecule database merge with experimental MS/MS data for METASPACE.

This script merges a molecule databae with experimental MS/MS data.  These data can then be submitted to METASPACE to search for nuetral loss and in-source fragments within datasets.

See: /Users/dis/PycharmProjects/word2vec/main_functions.py

In [3]:
# Load reference database by path:
path = '/Users/dis/PycharmProjects/core_metabolome/core_metabolome_v3.pickle'
ref_db = load_molecule_database(path)

In [4]:
# Parses GNPS_json to GNPS_df:
GNPS_json = '/Users/dis/PycharmProjects/word2vec/ALL_GNPS.json'
GNPS_df = parse_GNPS_json(GNPS_json)

In [494]:
# GNPS lipid rescue (not complete)
GNPS_json = '/Users/dis/PycharmProjects/word2vec/ALL_GNPS.json'
with open(GNPS_json, "r") as read_file:
    GNPS_data = json.load(read_file)
    GNPS_df = pd.DataFrame(GNPS_data)

In [ ]:
good_libs = ['GNPS-LIBRARY', 'GNPS-EMBL-MCF',
             'MMV_POSITIVE', 'MMV_NEGATIVE', 
             'LDB_POSITIVE', 'LDB_NEGATIVE',
             'GNPS-NIST14-MATCHES', 'GNPS-COLLECTIONS-MISC',
             'GNPS-MSMLS', 'BILELIB19', 
             'PNNL-LIPIDS-POSITIVE', 'PNNL-LIPIDS-NEGATIVE', 
             'MIADB', 'MASSBANK', 'MASSBANKEU', 'MONA', 
             'RESPECT', 'HMDB', 'CASMI', 'SUMNER']

In [498]:
# Identifies subset of GNPS with experimental specrta in ref db:
# Result: # 3920 spectra for 508 can_no_stereo_smiles
GNPS_hits_df = search_GNPS_targets(ref_db, GNPS_df)

In [502]:
GNPS_hits_df.library_membership.value_counts()

MASSBANK                                           1640
RESPECT                                            1369
HMDB                                               1269
GNPS-MSMLS                                          469
BILELIB19                                           441
GNPS-EMBL-MCF                                       258
GNPS-LIBRARY                                        115
SUMNER                                               95
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE        41
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE      25
MASSBANKEU                                           18
GNPS-NIST14-MATCHES                                  15
CASMI                                                14
LDB_NEGATIVE                                         11
GNPS-NIH-CLINICALCOLLECTION1                         10
GNPS-SELLECKCHEM-FDA-PART2                           10
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE       9
LDB_POSITIVE                                    

In [6]:
GNPS_hits_df.Adduct.value_counts()

M+H        1982
M-H         923
M+Na        464
M+K         219
M+NH4+      176
M-H2O+H     135
M+NH4        13
M+            6
M+Cl          1
M-H2O-H       1
Name: Adduct, dtype: int64

In [7]:
# Identifies subset of Mona with experimental specrta in ref db, runs in a few hours
# Result: # 7646 spectra for 648 inchi matches
# Mona_hits_df = search_MONA(ref_db)

# Reload existing:
Mona_hits_df = pd.read_pickle('mona_2020_Apr_15.pickle')

# Filters Mona_hits_df for high-res instruments, good adducts, and <20 ppm error.
# Result: # 3437 spectra for 535 inchi matches
Mona_hits_df = parse_MONA_out(Mona_hits_df)

In [8]:
Mona_hits_df.adduct.value_counts()

M+H        2805
M+Na        254
M+K         153
M+NH4+      142
M-H2O+H      56
M+           27
Name: adduct, dtype: int64

In [9]:
Mona_hits_df.source.value_counts()

MassBank                                   1197
Vaniya/Fiehn Natural Products Library       836
unknown                                     471
Fiehn HILIC Library                         379
GNPS                                        296
ReSpect                                     199
RIKEN PlaSMA Authentic Standard Library      59
Name: source, dtype: int64

In [22]:
# Cleans up ref db for only entries with experimental MS/MS spectra
df = preparser_Sirius(ref_db, GNPS_hits_df, Mona_hits_df)

In [ ]:
# Runs main Sirius loop
sirius_output_df = loop_Sirius(df, Mona_hits_df, GNPS_hits_df)

In [24]:
# Reload without rerunning
# sirius_output_df.to_pickle('sirius_output_df.pickle')
sirius_output_df = pd.read_pickle('sirius_output_df.pickle')

In [33]:
# Clean-up and merge output with MS2 spectra
clean_output_df = results_clean_up(df, sirius_output_df)

In [ ]:
# Clean-up database for METASPACE
METASPACE_db = output_METASPACE(clean_output_df, out_name)

In [464]:
# Export entire db
METASPACE_db.iloc[:,0:4].to_csv('whole_body_msms_test_v2.csv', sep='\t')

In [ ]:
# Filter for observed MS1 and export:
f2_ids = list(pd.read_csv('f2.txt').moleculeIds)
METASPACE_db = METASPACE_db[METASPACE_db.old_id.isin(f2_ids)]
METASPACE_db.iloc[:,0:4].to_csv('whole_body_msms_test_v3.csv', sep='\t')

In [491]:
# Before filtering 10019 fragments for 776 parents
# After fitering 1228 fragments for 111 parents 

In [527]:
def split_data_frame_list(df, target_column):
    # Accepts a column with multiple types and splits list variables to several rows.

    row_accumulator = []

    def split_list_to_rows(row):
        split_row = row[target_column]

        if isinstance(split_row, list):

          for s in split_row:
              new_row = row.to_dict()
              new_row[target_column] = s
              row_accumulator.append(new_row)

          if split_row == []:
              new_row = row.to_dict()
              new_row[target_column] = None
              row_accumulator.append(new_row)

        else:
          new_row = row.to_dict()
          new_row[target_column] = split_row
          row_accumulator.append(new_row)

    df.apply(split_list_to_rows, axis=1)
    new_df = pd.DataFrame(row_accumulator)

    return new_df

In [577]:
def a_label(x, a):
    # Finds string a in string x
    if a in x:
        return 1
    else:
        return 0

In [626]:
# Read METASPACE output
# Generated at 4/17/2020 12:08:16 PM. For help see https://bit.ly/2HO2uz4
# URL: https://metaspace2020.eu/annotations?db=whole_body_MSMS_test_v3&prj=a493c7b8-e27f-11e8-9d75-3bb2859d3748&ds=2017-05-17_19h49m04s&fdr=0.5&sort=-mz&hideopt=1&sections=3&page=7

ms_out = pd.read_csv('metaspace_annotations.csv', header=2)
ms_out = ms_out[ms_out.adduct == 'M[M]+']
ms_out['n_ids_formula'] = ms_out.moleculeIds.apply(lambda x: len(x.split(',')))
gc = ['datasetId', 'formula', 'adduct', 'mz', 'fdr',
      'moleculeNames']
ms_out = ms_out[gc]
ms_out['moleculeNames'] = ms_out['moleculeNames'].apply(lambda x: x.split(', '))
ms_out = split_data_frame_list(ms_out, 'moleculeNames')
ms_out['moleculeNames'] = ms_out['moleculeNames'].apply(lambda x: x.split('_',2))
df = pd.DataFrame(ms_out['moleculeNames'].tolist(), columns=['id', 'par_frag', 'name'])
ms_out = pd.concat([df, ms_out, ], axis=1)
ms_out = ms_out.sort_values(by=['name'])

# Label with number of parents and fragments for each id
ms_out['parent'] = ms_out['par_frag'].apply(lambda x: a_label(x, 'p'))
ms_out['n_frag'] = ms_out['par_frag'].apply(lambda x: a_label(x, 'f'))
df = ms_out[['id', 'parent', 'n_frag']]
df = df.groupby('id').sum()
ms_out = ms_out.merge(df, on='id', how='left')
gc = ['id', 'par_frag', 'name', 'datasetId', 'formula',
       'adduct', 'mz', 'fdr', 'moleculeNames',
       'parent_y', 'n_frag_y']
ms_out = ms_out[gc].copy(deep=True)

# Label with number of degenerate formulas at each mass
df = ms_out[['id', 'formula']]
df = df.groupby('formula').nunique()
df = df.iloc[:,0:1]
ms_out = ms_out.merge(df, on='formula', how='left')

In [629]:
ms_out.to_pickle('ms_out_v3_test01.pickle')

In [630]:
ms_out

,id_x,par_frag,name,datasetId,formula,adduct,mz,fdr,moleculeNames,parent_y,n_frag_y,id_y
0,HMDB0003601,12f,(S)-Reticuline,2017-05-17_19h49m04s,C9H11O2,M[M]+,151.075336,0.20,"[HMDB0003601, 12f, (S)-Reticuline]",0,10,2
1,HMDB0003601,18f,(S)-Reticuline,2017-05-17_19h49m04s,C10H12NO2,M[M]+,178.086235,0.50,"[HMDB0003601, 18f, (S)-Reticuline]",0,10,1
2,HMDB0003601,19f,(S)-Reticuline,2017-05-17_19h49m04s,C11H14NO2,M[M]+,192.101885,0.50,"[HMDB0003601, 19f, (S)-Reticuline]",0,10,1
3,HMDB0003601,14f,(S)-Reticuline,2017-05-17_19h49m04s,C10H10NO,M[M]+,160.075671,0.05,"[HMDB0003601, 14f, (S)-Reticuline]",0,10,1
4,HMDB0003601,21f,(S)-Reticuline,2017-05-17_19h49m04s,C15H13O,M[M]+,209.096072,0.50,"[HMDB0003601, 21f, (S)-Reticuline]",0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...
301,HMDB0000960,20f,dGDP,2017-05-17_19h49m04s,C7H8N2O2,M[M]+,152.058009,0.05,"[HMDB0000960, 20f, dGDP]",1,3,4
302,HMDB0000960,22f,dGDP,2017-05-17_19h49m04s,C10H15N4O3,M[M]+,239.113847,0.20,"[HMDB0000960, 22f, dGDP]",1,3,4
303,HMDB0001409,3p,dUMP,2017-05-17_19h49m04s,C9H14N2O8P,M[M]+,309.048209,0.50,"[HMDB0001409, 3p, dUMP]",1,0,1
304,HMDB0000211,1p,myo-Inositol,2017-05-17_19h49m04s,C6H12O6Na,M[M]+,203.052590,0.05,"[HMDB0000211, 1p, myo-Inositol]",1,0,24


# Friday:
1. API access ion images and put in folders as pairs
2. Add libraries HMDB, all Mona dl, PNNL in GNPS

Next Jupyter notebook:
http://localhost:8888/notebooks/PycharmProjects/word2vec/export_ion_mages.ipynb